<a href="https://colab.research.google.com/github/Rimsha996/UKC-thesis/blob/main/K_fold_DensetNet121_2Cat_(1).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# @title
google_colab_flag = True # Make it False if do not intend to use Google Colab and want to train in local machine!!

# For training in Google Colab
if(google_colab_flag):
  from google.colab import drive
  drive.mount('/content/drive')
  !ls
  import sys
  root_path = '/content/drive/MyDrive/FlippedImages -removedBlackBorder/Data-resized(300,400)-2Categories - Copy/' # This is the path to where in google drive the code is stored!
  sys.path.append(root_path)

# For local training
else:
  root_path = ''

Mounted at /content/drive
drive  sample_data


In [ ]:
import numpy as np
from sklearn.metrics import accuracy_score, f1_score, precision_score, confusion_matrix
from sklearn.model_selection import StratifiedKFold
from PIL import Image
import random
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession
import warnings
import os
import shutil
from PIL import ImageFile
warnings.simplefilter('error', Image.DecompressionBombWarning)
ImageFile.LOAD_TRUNCATED_IMAGES = True
from PIL import Image
Image.MAX_IMAGE_PIXELS = 100000000
config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint
datasetFolderName=root_path
MODEL_FILENAME=root_path+"model_DenseNet_2_cv.h5"

checkpoint = ModelCheckpoint(MODEL_FILENAME, save_best_only=True, save_weights_only=True, verbose=1 )
callbacks_list = [checkpoint]

sourceFiles=[]
classLabels=['mild','severe']
X=[]
Y=[]

img_rows, img_cols =  400,300 # input image dimensions
train_path=datasetFolderName+'/train/'
validation_path=datasetFolderName+'/val/'
test_path=datasetFolderName+'/test/'

In [ ]:
def transferBetweenFolders(source, dest, splitRate):
    global sourceFiles
    sourceFiles=os.listdir(source)
    if(len(sourceFiles)!=0):
        transferFileNumbers=int(len(sourceFiles)*splitRate)
        transferIndex=random.sample(range(0, len(sourceFiles)), transferFileNumbers)
        for eachIndex in transferIndex:
            shutil.move(source+str(sourceFiles[eachIndex]), dest+str(sourceFiles[eachIndex]))
    else:
        print("No file moved. Source empty!")

def transferAllClassBetweenFolders(source, dest, splitRate):
    for label in classLabels:
        transferBetweenFolders(datasetFolderName+'/'+source+'/'+label+'/',
                               datasetFolderName+'/'+dest+'/'+label+'/',
                               splitRate)

def my_metrics(y_true, y_pred):
    accuracy=accuracy_score(y_true, y_pred)
    precision=precision_score(y_true, y_pred,average='weighted')
    f1Score=f1_score(y_true, y_pred, average='weighted')
    print("Accuracy  : {}".format(accuracy))
    print("Precision : {}".format(precision))
    print("f1Score : {}".format(f1Score))
    cm=confusion_matrix(y_true, y_pred)
    print(cm)
    return accuracy, precision, f1Score

In [ ]:
transferAllClassBetweenFolders('test', 'train', 1.0)

In [ ]:
transferAllClassBetweenFolders('train', 'test', 0.30)

In [ ]:
def prepareNameWithLabels(folderName):
    sourceFiles=os.listdir(datasetFolderName+'/train/'+folderName)
    for val in sourceFiles:
        X.append(val)
        for i in range(len(classLabels)):
          if(folderName==classLabels[i]):
              Y.append(i)

In [ ]:
# Organize file names and class labels in X and Y variables
for i in range(len(classLabels)):
  prepareNameWithLabels(classLabels[i])

In [ ]:
print(X)
print(Y)

['G1IMG0.jpg', 'G1IMG4.jpg', 'G1IMG1.jpg', 'G1IMG5.jpg', 'G1IMG6.jpg', 'G1IMG3.jpg', 'G1IMG9.jpg', 'G1IMG19.jpg', 'G1IMG26.jpg', 'G1IMG14.jpg', 'G1IMG18.jpg', 'G1IMG12.jpg', 'G1IMG13.jpg', 'G1IMG7.jpg', 'G1IMG27.jpg', 'G1IMG21.jpg', 'G1IMG16.jpg', 'G1IMG17.jpg', 'G1IMG8.jpg', 'G1IMG10.jpg', 'G1IMG15.jpg', 'G2IMG10.jpg', 'G2IMG12.jpg', 'G2IMG1.jpg', 'G2IMG9.jpg', 'G2IMG13.jpg', 'G2IMG19.jpg', 'G2IMG7.jpg', 'G2IMG11.jpg', 'G2IMG21.jpg', 'G2IMG6.jpg', 'G2IMG25.jpg', 'G2IMG8.jpg', 'G2IMG18.jpg', 'G2IMG26.jpg', 'G2IMG22.jpg', 'G2IMG5.jpg', 'G2IMG15.jpg', 'G2IMG20.jpg', 'G2IMG2.jpg', 'mild_original_G1IMG0.jpg_b5174d64-975b-4fba-911b-861bc5da2f49.jpg', 'mild_original_G2IMG27.jpg_27b605b1-e6f8-485e-ac00-12af88739956.jpg', 'mild_original_G2IMG6.jpg_52f45692-06e0-48ea-b483-8a029067c3af.jpg', 'mild_original_G1IMG14.jpg_bce39c07-a52a-40fb-b9ab-47d0440ba3e1.jpg', 'mild_original_G1IMG12.jpg_260b9b89-f51e-4fff-9193-3839378195b0.jpg', 'mild_original_G2IMG5.jpg_3e586b74-555a-4182-b699-877b5e37a5dc.jpg'

In [ ]:
X=np.asarray(X)
Y=np.asarray(Y)

In [ ]:
from tensorflow.keras.applications import DenseNet121
from tensorflow.keras import models

conv_base=DenseNet121(weights='imagenet',include_top=False,input_shape=(300,400,3))

29097984/29084464 [==============================] - 0s 0us/step


In [ ]:
# Note that, this model structure is a very basic one. To achieve better performance, you should change the model structure and hyperparameters according to your needs and data.
batch_size = 16
epoch=50
def getModel():
    model = models.Sequential()
    model.add(conv_base)
    model.add(layers.Flatten())
    model.add(layers.Dense(64, activation='relu'))
    model.add(Dense(len(classLabels), activation='softmax'))
    # model.compile(loss='categorical_crossentropy',optimizer=optimizers.Adagrad(),metrics=['accuracy'])
    model.compile(optimizer='adagrad', loss='categorical_crossentropy', metrics=['accuracy'])

    return model

model=getModel()

In [ ]:
import math

# ===============Stratified K-Fold======================
skf = StratifiedKFold(n_splits=3, shuffle=True)
skf.get_n_splits(X, Y)
foldNum=0

for train_index, val_index in skf.split(X, Y):
    #First cut all images from validation to train (if any exists)
    transferAllClassBetweenFolders('val', 'train', 1.0)
    foldNum+=1
    print("Results for fold",foldNum)
    X_train, X_val = X[train_index], X[val_index]
    Y_train, Y_val = Y[train_index], Y[val_index]
    # Move validation images of this fold from train folder to the validation folder
    for eachIndex in range(len(X_val)):
        classLabel=''
        for i in range(len(classLabels)):
          if(Y_val[eachIndex]==i):
              classLabel=classLabels[i]
        #Then, copy the validation images to the validation folder
        shutil.move(datasetFolderName+'/train/'+classLabel+'/'+X_val[eachIndex],
                    datasetFolderName+'/val/'+classLabel+'/'+X_val[eachIndex])

    train_datagen = ImageDataGenerator(
                      rescale=1./255,
                      rotation_range=40,
                      width_shift_range=0.2,
                      height_shift_range=0.2,
                      shear_range=0.2,
                      zoom_range=0.2)
    validation_datagen = ImageDataGenerator(rescale=1./255)
    test_datagen = ImageDataGenerator(rescale=1./255)

    #Start ImageClassification Model
    train_generator = train_datagen.flow_from_directory(
        train_path,
        target_size=(img_rows, img_cols),
        batch_size=batch_size,
        class_mode='categorical',
        subset='training')

    validation_generator = validation_datagen.flow_from_directory(
        validation_path,
        target_size=(img_rows, img_cols),
        batch_size=batch_size,
        class_mode=None,  # only data, no labels
        shuffle=False)

    # fit model
    history=model.fit_generator(train_generator,
                        steps_per_epoch=math.ceil(len(X_train)//batch_size),
                        # callbacks = callbacks_list,
                        epochs=epoch)
    # history = model.fit_generator(
    # train_generator ,
    # steps_per_epoch=math.ceil(X_train//batch_size),
    # epochs=epoch,
    # callbacks=callbacks_list,
    # validation_data=validation_generator,
    # validation_steps=math.ceil(number_of_test_samples//batch_size)
    # )

    predictions = model.predict_generator(validation_generator, verbose=1)
    yPredictions = np.argmax(predictions, axis=1)
    true_classes = validation_generator.classes
    # evaluate validation performance
    print("***Performance on Validation data***")
    valAcc, valPrec, valFScore = my_metrics(true_classes, yPredictions)

Results for fold 1
Found 201 images belonging to 2 classes.
Found 67 images belonging to 2 classes.


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:54: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.


Epoch 1/50
8/8 [==============================] - 232s 27s/step - loss: 2.5711 - accuracy: 0.5124
Epoch 2/50
8/8 [==============================] - 212s 26s/step - loss: 0.6575 - accuracy: 0.6116
Epoch 3/50
8/8 [==============================] - 215s 27s/step - loss: 0.5574 - accuracy: 0.7190
Epoch 4/50
8/8 [==============================] - 223s 28s/step - loss: 0.5697 - accuracy: 0.7031
Epoch 5/50
8/8 [==============================] - 210s 26s/step - loss: 0.4541 - accuracy: 0.7851
Epoch 6/50
8/8 [==============================] - 225s 28s/step - loss: 0.5231 - accuracy: 0.7500
Epoch 7/50
8/8 [==============================] - 225s 28s/step - loss: 0.5159 - accuracy: 0.7266
Epoch 8/50
8/8 [==============================] - 237s 30s/step - loss: 0.4595 - accuracy: 0.7578
Epoch 9/50
8/8 [==============================] - 240s 30s/step - loss: 0.4379 - accuracy: 0.8047
Epoch 10/50
8/8 [==============================] - 240s 30s/step - loss: 0.4209 - accuracy: 0.8047
Epoch 11/50
8/8 [==

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:64: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.


5/5 [==============================] - 21s 4s/step
***Performance on Validation data***
Accuracy  : 0.9402985074626866
Precision : 0.9417238133656044
f1Score : 0.9401647675600493
[[34  1]
 [ 3 29]]
Results for fold 2
Found 201 images belonging to 2 classes.
Found 67 images belonging to 2 classes.


...

5/5 [==============================] - 18s 3s/step
***Performance on Validation data***
Accuracy  : 1.0
Precision : 1.0
f1Score : 1.0
[[35  0]
 [ 0 32]]
Results for fold 3
Found 201 images belonging to 2 classes.
Found 67 images belonging to 2 classes.


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:54: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.


Epoch 1/50
8/8 [==============================] - 175s 22s/step - loss: 0.0312 - accuracy: 1.0000
Epoch 2/50
8/8 [==============================] - 185s 23s/step - loss: 0.0222 - accuracy: 1.0000
Epoch 3/50
8/8 [==============================] - 185s 23s/step - loss: 0.0189 - accuracy: 0.9922
Epoch 4/50
8/8 [==============================] - 175s 22s/step - loss: 0.0384 - accuracy: 0.9917
Epoch 5/50
8/8 [==============================] - 185s 23s/step - loss: 0.0162 - accuracy: 1.0000
Epoch 6/50
8/8 [==============================] - 175s 22s/step - loss: 0.0269 - accuracy: 0.9917
Epoch 7/50
8/8 [==============================] - 175s 22s/step - loss: 0.0158 - accuracy: 1.0000
Epoch 8/50
8/8 [==============================] - 175s 22s/step - loss: 0.0240 - accuracy: 0.9917
Epoch 9/50
8/8 [==============================] - 186s 23s/step - loss: 0.0231 - accuracy: 0.9844
Epoch 10/50
8/8 [==============================] - 186s 23s/step - loss: 0.0376 - accuracy: 0.9844
Epoch 11/50
8/8 [==

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:64: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.


5/5 [==============================] - 19s 4s/step
***Performance on Validation data***
Accuracy  : 1.0
Precision : 1.0
f1Score : 1.0
[[35  0]
 [ 0 32]]


In [ ]:
print("==============TEST RESULTS============")
test_generator = test_datagen.flow_from_directory(
        test_path,
        target_size=(img_rows, img_cols),
        batch_size=batch_size,
        class_mode=None,
        shuffle=False)
predictions = model.predict(test_generator, verbose=1)
yPredictions = np.argmax(predictions, axis=1)
true_classes = test_generator.classes

testAcc,testPrec, testFScore = my_metrics(true_classes, yPredictions)
model.save(MODEL_FILENAME)

==============TEST RESULTS============
Found 56 images belonging to 2 classes.
4/4 [==============================] - 21s 4s/step
Accuracy  : 1.0
Precision : 1.0
f1Score : 1.0
[[28  0]
 [ 0 28]]
